Let's start by creating an instance of the droneMPC class. Initializing in hover and r=l0. Set a reference and call the solve method to see if everything runs through.

In [ ]:
from drone_MPC import DroneMPC, DroneMPCOptions
import numpy as np

# %% Hovering test for DroneMPC

# Create an instance of the MPC options (using the updated physical parameters)
opts = DroneMPCOptions(
    M=2.0,         # drone mass in kg
    m=0.1,         # load mass in kg (small compared to drone)
    Ixx=0.05,      # moment of inertia in kg·m²
    g=9.81,        # gravitational acceleration in m/s²
    c=1.0,         # rotor thrust constant
    L_rot=0.2,     # half distance between rotors in m
    k=10.0,       # stiff spring (N/m)
    l0=0.3,        # spring rest length in m
    # Constraints and cost weighting matrices remain as defined by default
)

# To test the full pendulum model, set switch_stage equal or above N:
opts.switch_stage = opts.N  # use the full model over the entire horizon

# Create an instance of the DroneMPC class
mpc = DroneMPC(opts)

# Define an initial state corresponding to hovering.
# For the full model, the state is:
# [y, z, phi, r, theta, y_dot, z_dot, phi_dot, r_dot, theta_dot, w1, w2]
# where:
#   - y = 0.0 (horizontal position), z = 1.0 (altitude)
#   - phi (drone pitch) = 0.0, theta (pendulum angle) = 0.0
#   - r is set to the spring rest length (l0)
#   - All velocities are 0.
#   - Rotor speeds (w1, w2) are chosen such that the total thrust equals the weight.
l0 = opts.l0
M_total = opts.M + opts.m
# Required total thrust T = (M_total)*g, and assuming symmetric rotors, each rotor speed is set to T/2 (with c=1)
w_val = (M_total * opts.g) / 2.0

x0 = np.array([
    0.0,    # y position
    1.0,    # z position (hover altitude)
    0.0,    # phi (drone pitch)
    l0,     # r (pendulum length set to rest length)
    0.0,    # theta (pendulum angle)
    0.0,    # y_dot
    0.0,    # z_dot
    0.0,    # phi_dot
    0.0,    # r_dot
    0.0,    # theta_dot
    w_val,  # w1 (rotor speed)
    w_val   # w2 (rotor speed)
])
print("Initial hovering state:", x0)

# Define a reference position far away (for example, [10, 10])
pos_ref = np.array([10.0, 10.0])

# Call the MPC solve function to compute the first control input
u0 = mpc.solve(x0, pos_ref)
print("First control input:", u0)

/home/lukas/ETH_Zuerich/ASL_Stanford_Masterthesis/Workspace/multi_fidelity_MPC_fast_slow/drone/drone_MPC.py:426: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if stage is not 0 and stage == self.opts.switch_stage: # 0 stage means only 2d model and no switch



got cost_type EXTERNAL for cost_type_e, hessian_approx: 'GAUSS_NEWTON'.
With this setting, acados will proceed computing the exact Hessian for the cost term and no Hessian contribution from constraints and dynamics.
If the external cost is a linear least squares cost, this coincides with the Gauss-Newton Hessian.
Note: There is also the option to use the external cost module with a numerical Hessian approximation (see `ext_cost_num_hess`).
OR the option to provide a symbolic custom Hessian approximation (see `cost_expr_ext_cost_custom_hess`).


Error while creating function drone_full_1742860456875_cost_y_e_fun with inputs [MX(x), MX(0x0), MX(0x0), [], MX(0x0)] and outputs [MX(@1=0.2, @2=0.1, @3=(@2*x[10]), @4=(x[2]-x[4]), @5=sin(@4), @6=0.4, @7=(@6*x[8]), @8=x[9], @9=0.1, @10=(@9*x[11]), @11=(x[2]-x[4]), @12=sin(@11), @13=(@1*x[3]), @14=((((@3*@5)-(@7*@8))+(@10*@12))/@13), @15=x[4], @16=cos(@15), @17=x[3], @18=(((zeros(12x1)[4] += (@16*@17))[3] += sin(@15))[0] += 1), @19=((dense((((z

RuntimeError: Error in Function::Function for 'drone_full_1742860456875_cost_y_e_fun' [MXFunction] at .../casadi/core/function.cpp:280:
.../casadi/core/function_internal.cpp:146: Error calling MXFunction::init for 'drone_full_1742860456875_cost_y_e_fun':
.../casadi/core/mx_function.cpp:406: drone_full_1742860456875_cost_y_e_fun::init: Initialization failed since variables [u] are free. These symbols occur in the output expressions but you forgot to declare these as inputs. Set option 'allow_free' to allow free variables.